<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Project Title: Machine Translation With HuggingFace Transformers English To Arabic Using LLM</b></h1>



### Tabel of Contents:



* [Import Libraries](#1)

* [Load Dataset](#2)

* [MBart Model](#4)

* [Evaluation](#3)

* [Predictive for Test](#8)

* [Like this? Upvote and comment! 🌊 End](#6)

**<a id="1"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Import Libraries</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [1]:
!pip install datasets

In [2]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset,DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,Seq2SeqTrainer,Seq2SeqTrainingArguments
from huggingface_hub import login
import wandb
wandb.init(mode = "disabled")
import warnings
warnings.filterwarnings('ignore')

**<a id="2"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Load Dataset</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [36]:
dataset = load_dataset("akbargherbal/10K_english_to_arabic_dataset_for_FT")
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 10000
    })
})

In [37]:
#  split to train 70 test 15 validation 15 error
train_testvalid = dataset["train"].train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['train'],
    'validation': test_valid['test']
})

dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 1000
    })
})

In [38]:
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
validation_df = pd.DataFrame(dataset['validation'])
# Select columns 0 and 1 for test and validation DataFrames
train_df = test_df.iloc[:, :2]
test_df = test_df.iloc[:, :2]
validation_df = validation_df.iloc[:, :2]
# Rename columns in test and validation DataFrames using train DataFrame's column names
train_df.columns = ['ar','en']
test_df.columns = ['ar','en']
validation_df.columns = ['ar','en']
train_df.head()

,ar,en
0,68 - وإنني على ثقة من أن العلاقات بين تيمور - ...,68. I am confident that relations between Timo...
1,كما أكدت الجمعية العامة التوجه الرئيسي لهذه ال...,The thrust of the vision was further confirmed...
2,بيد أن جميع القرارات الخاصة بالمضمون تقتضي أن ...,"However, all decisions on substance would requ..."
3,(ب) رفع مستوى إشراك الموظفين بهدف جعل صياغة بر...,Improved engagement of staff with the aim of m...
4,الرئيس (متحدثاً بالإنكليزية): أشكر السفير سود ...,The PRESIDENT: I thank Ambassador Sood for his...


In [39]:
test_df.head()

,ar,en
0,68 - وإنني على ثقة من أن العلاقات بين تيمور - ...,68. I am confident that relations between Timo...
1,كما أكدت الجمعية العامة التوجه الرئيسي لهذه ال...,The thrust of the vision was further confirmed...
2,بيد أن جميع القرارات الخاصة بالمضمون تقتضي أن ...,"However, all decisions on substance would requ..."
3,(ب) رفع مستوى إشراك الموظفين بهدف جعل صياغة بر...,Improved engagement of staff with the aim of m...
4,الرئيس (متحدثاً بالإنكليزية): أشكر السفير سود ...,The PRESIDENT: I thank Ambassador Sood for his...


In [40]:
validation_df.head()

,ar,en
0,ويذكر أن المسؤولية المزعومة عن اﻻنتهاكات التي ...,The presumption of responsibility for the conf...
1,وقامت أمانة الفريق، بوجه خاص، بتنسيق بيان شفوي...,"In particular, the Panel secretariat coordinat..."
2,كما أنه يتولى المسؤولية عن إدارة شؤون الموظفين...,He is also responsible for the administration ...
3,71- وفي الختام، ذكر الوفد أن هناك مسائل كثيرة ...,"71. In conclusion, the delegation stated that ..."
4,11 - وتحيط مجموعة فيينا علما بالقرار الذي اتخذ...,11. The Vienna Group notes the decision taken ...


In [41]:
train_df["en"] = train_df["en"].astype(str)
train_df["ar"] = train_df["ar"].astype(str)
test_df["en"] = test_df["en"].astype(str)
test_df["ar"] = test_df["ar"].astype(str)
validation_df["en"] = validation_df["en"].astype(str)
validation_df["ar"] = validation_df["ar"].astype(str)

In [42]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)
train_dataset

Dataset({
    features: ['ar', 'en'],
    num_rows: 1000
})

In [43]:
test_dataset

Dataset({
    features: ['ar', 'en'],
    num_rows: 1000
})

<a id="4"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">MBart Model</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [44]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50")

In [45]:
input_max_len = max(len(tokenizer.encode(text)) for text in train_dataset['en'])
output_max_len = max(len(tokenizer.encode(text)) for text in train_dataset['ar'])
print(f"Calculated Input max_length: {input_max_len}")
print(f"Calculated Output max_length: {output_max_len}")

Calculated Input max_length: 245
Calculated Output max_length: 311


In [46]:
# Tokenization function
def tokenize_function(examples):
    # Tokenize the en and ar
    inputs = tokenizer(examples["en"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples["ar"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
validation_dataset = validation_dataset.map(tokenize_function, batched=True)
print(train_dataset[0])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'ar': '68 - وإنني على ثقة من أن العلاقات بين تيمور - ليشتي وإندونيسيا سوف تزداد توطدا بفضل الجهود المشتركة بين القيادات التيمورية والحكومة الإندونيسية الجديدة.', 'en': '68. I am confident that relations between Timor-Leste and Indonesia will further strengthen with the joint efforts of the Timorese leaders and the new Government of Indonesia.', 'input_ids': [250004, 12045, 5, 87, 444, 158863, 450, 91582, 17721, 51189, 42, 9, 866, 4896, 136, 3799, 1221, 53333, 90254, 33, 678, 70, 124416, 79825, 111, 70, 51189, 61776, 119893, 136, 70, 3525, 92416, 111, 3799, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [47]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./Finetuning_MBart_English_Arabic_Translation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

In [48]:
# Train the model
trainer.train()
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,No log,1.353817


In [49]:
tokenizer.save_pretrained(training_args.output_dir)

('./Finetuning_MBart_English_Arabic_Translation/tokenizer_config.json',
 './Finetuning_MBart_English_Arabic_Translation/special_tokens_map.json',
 './Finetuning_MBart_English_Arabic_Translation/sentencepiece.bpe.model',
 './Finetuning_MBart_English_Arabic_Translation/added_tokens.json',
 './Finetuning_MBart_English_Arabic_Translation/tokenizer.json')

In [ ]:
login(token="HUGGINGFACE_TOKEN")

In [51]:
repo_name = "ahmed792002/Finetuning_MBart_English_Arabic_Translation"
trainer.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

events.out.tfevents.1733434661.947342abf294.9988.0:   0%|          | 0.00/6.66k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ahmed792002/Finetuning_MBart_English_Arabic_Translation/commit/f85846648adb8edb1ce3973dd0407a738f6e487e', commit_message='Upload tokenizer', commit_description='', oid='f85846648adb8edb1ce3973dd0407a738f6e487e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ahmed792002/Finetuning_MBart_English_Arabic_Translation', endpoint='https://huggingface.co', repo_type='model', repo_id='ahmed792002/Finetuning_MBart_English_Arabic_Translation'), pr_revision=None, pr_num=None)

**<a id="1"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Evaluation</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [52]:
results = trainer.evaluate(test_dataset)
print("Evaluation results:")
print("Test Loss",results["eval_loss"])

Evaluation results:
Test Loss 1.22391939163208


<a id="8"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 20px 10px 10px #042b4c">Predictive for Test</p></b></h1>

In [53]:
tokenizer = AutoTokenizer.from_pretrained("ahmed792002/Finetuning_MBart_English_Arabic_Translation")
model = AutoModelForSeq2SeqLM.from_pretrained("ahmed792002/Finetuning_MBart_English_Arabic_Translation")

tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [61]:
def translate_text(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    # Generate translation
    outputs = model.generate(inputs["input_ids"], num_beams=4, early_stopping=True)
    # Decode the translation
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

In [62]:
test_sentence = test_dataset["en"][50]
test_summary = test_dataset["ar"][50]
translation = translate_text(test_sentence)
# Display results
print(f"Original :\n{test_sentence}")
print("\n","*"*50,"\n")
print(f"Original Translation :\n{test_summary}")
print("\n","*"*50,"\n")
print(f"Translation :\n{translation}")

Original :
V. Population policies 26. World Population Policies, 20034 was published during 2004.

 ************************************************** 

Original Translation :
26 - نُشر مجلد السياسات السكانية في العالم لعام 2003(4) خلال عام 2004.

 ************************************************** 

Translation :
V. السياسة السكانية 26. السياسة السكانية العالمية، 20034، نشر في عام 2004.


In [63]:
test_sentence = test_dataset["en"][200]
test_summary = test_dataset["ar"][200]
translation = translate_text(test_sentence)
# Display results
print(f"Original :\n{test_sentence}")
print("\n","*"*50,"\n")
print(f"Original Translation :\n{test_summary}")
print("\n","*"*50,"\n")
print(f"Translation :\n{translation}")

Original :
We also believe that although some responses might not meet every Member State &apos; s expectation, they could act as catalysts and springboards for new approaches and new solutions in Durban.

 ************************************************** 

Original Translation :
ونعتقد أيضا أنه على الرغم من أن بعض الاستجابات قد لا تلبي توقعات كل دولة عضو، فإنها يمكن أن تكون حافزا ومنطلقا نحو نُهُج جديدة وحلول جديدة في ديربان.

 ************************************************** 

Translation :
وأشارت إلى أن بعض الاقتراحات قد لا تفي بالتوقعات التي يتوقعها كل من الدول الأعضاء، ولكنها يمكن أن تكون حافزاً وحافزاً لطرح حلول جديدة في بروكسل.


In [64]:
test_sentence = test_dataset["en"][800]
test_summary = test_dataset["ar"][800]
translation = translate_text(test_sentence)
# Display results
print(f"Original :\n{test_sentence}")
print("\n","*"*50,"\n")
print(f"Original Translation :\n{test_summary}")
print("\n","*"*50,"\n")
print(f"Translation :\n{translation}")

Original :
430. The Trust Fund was established by General Assembly resolution 64/71.

 ************************************************** 

Original Translation :
430 - أنشئ هذا الصندوق بموجب قرار الجمعية العامة 64/71.

 ************************************************** 

Translation :
430 - تم إنشاء صندوق صدقات بموجب قرار الجمعية العامة 64/71.


In [65]:
test_sentence = """
Hello! My name is Ahmed Ashraf.
I am a Data Scientist with a Bachelor's degree in Artificial Intelligence from Benha University, graduating with a GPA of 3.76. I have experience in machine learning, deep learning, and natural language processing through internships and various projects.

Key Highlights:
Worked on Alzheimer’s detection using deep learning models.
Built a real-time American Sign Language detection system with GANs.
Optimized large-scale sentiment analysis models for 3 million Amazon reviews.
I am passionate about solving challenging AI problems and sharing knowledge through teaching and projects. Let's innovate together!
"""
translation = translate_text(test_sentence)
# Display results
print(f"Original :\n{test_sentence}")
print("\n","*"*50,"\n")
print(f"Translation :\n{translation}")

Original :

Hello! My name is Ahmed Ashraf.
I am a Data Scientist with a Bachelor's degree in Artificial Intelligence from Benha University, graduating with a GPA of 3.76. I have experience in machine learning, deep learning, and natural language processing through internships and various projects.

Key Highlights:
Worked on Alzheimer’s detection using deep learning models.
Built a real-time American Sign Language detection system with GANs.
Optimized large-scale sentiment analysis models for 3 million Amazon reviews.
I am passionate about solving challenging AI problems and sharing knowledge through teaching and projects. Let's innovate together!


 ************************************************** 

Translation :
Hello! اسمي Ahmed Ashraf. أحمل درجة الماجستير في الذكاء الاصطناعي من جامعة بنغلاديش، وحصلت على معدل تراكمي 3.76 وشاركت في العديد من المشاريع. أهم الإنجازات: عملت على تشخيص مرض الزهايمر باستخدام التعلم الاصطناعي. صممت نظام تحليل الأصوات المدمج مع GAN. صممت نظام تحليل العاطفة

In [66]:
translation

'Hello! اسمي Ahmed Ashraf. أحمل درجة الماجستير في الذكاء الاصطناعي من جامعة بنغلاديش، وحصلت على معدل تراكمي 3.76 وشاركت في العديد من المشاريع. أهم الإنجازات: عملت على تشخيص مرض الزهايمر باستخدام التعلم الاصطناعي. صممت نظام تحليل الأصوات المدمج مع GAN. صممت نظام تحليل العاطفة المدمج مع GAN. صممت نظام تحليل العاطفة المدمج مع GAN.'

<center><span style="font-family:Palatino; font-size:22px;"><i>Like this? <span style="color:#DC143C;">Upvote and Comment!</span> </i>🌊 End</span> </center>